In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
# Define a simple time series dataset
sequence_length = 10
input_size = 1
hidden_size = 32

# Generate some random time series data
data = np.sin(np.linspace(0, 100, sequence_length))

# Convert the data into a PyTorch tensor
data = torch.FloatTensor(data).view(1, sequence_length, input_size)

In [3]:

# Define the RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    
    def forward(self, x):
        # Initialize the hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward pass
        out, _ = self.rnn(x, h0)
        return out

In [4]:
#Create an instance of the RNN model
num_layers = 1
model = SimpleRNN(input_size, hidden_size, num_layers)

# Apply the model to the data
output = model(data)

# Print the shape of the output
print("Output shape:", output.shape)

Output shape: torch.Size([1, 10, 32])
